In [133]:
import pandas as pd
import numpy as np

In [134]:
Movies = pd.read_csv("/home/aman/ML Projects/Movie Recommender System/tmdb_5000_movies.csv")
Credits = pd.read_csv("/home/aman/ML Projects/Movie Recommender System/tmdb_5000_credits.csv")

In [135]:
Movies = Movies.merge(Credits,on='title')

In [136]:
Movies.shape

(4809, 23)

In [137]:
Movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [138]:
Movies.drop(columns=['budget','homepage','revenue','original_language','original_title','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','tagline','vote_average','movie_id','status','vote_count'],inplace=True)

In [139]:
Movies.sample()

,genres,id,keywords,overview,title,cast,crew
1143,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",27936,"[{""id"": 156052, ""name"": ""unemployment""}, {""id""...",A man and his friends come up with an intricat...,Micmacs,"[{""cast_id"": 1, ""character"": ""Bazil"", ""credit_...","[{""credit_id"": ""52fe456ec3a368484e05a8cf"", ""de..."


In [140]:
Movies.isnull().sum()

genres      0
id          0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [141]:
Movies.dropna(inplace=True)

In [142]:
Movies.duplicated().sum()

0

In [143]:
Movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [144]:
import ast

In [145]:
def convert(value):
    list = []
    for i in ast.literal_eval(value):
        list.append(i['name'])
    return list

In [146]:
Movies['genres'] = Movies['genres'].apply(convert)

In [147]:
Movies.head()

,genres,id,keywords,overview,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [148]:
Movies['keywords']  = Movies['keywords'].apply(convert)

In [149]:
def convertcast(value):
    list = []
    counter = 0
    for i in ast.literal_eval(value):
        if counter != 3:
            list.append(i['name'])
            counter+=1
        else:
            break
    return list

In [150]:
Movies['cast'] = Movies['cast'].apply(convertcast)

In [151]:
def fetchdirec(value):
    list = []
    for i in ast.literal_eval(value):
        if i['job'] == 'Director':
            list.append(i['name'])
            break
    return list

In [152]:
Movies['crew'] = Movies['crew'].apply(fetchdirec)

In [153]:
Movies['overview'] = Movies['overview'].apply(lambda x:x.split())

In [154]:
Movies['genres'] = Movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['keywords'] = Movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['cast'] = Movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
Movies['crew'] = Movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [155]:
Movies['tags'] = Movies['overview'] + Movies['genres'] + Movies['keywords'] + Movies['cast'] + Movies['crew']

In [156]:
new_df = Movies[['id' , 'title' , 'tags']]

In [157]:
new_df.head()

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [158]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipykernel_6951/3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [159]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_6951/3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [160]:
import nltk

In [161]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [162]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [163]:
new_df['tags'] = new_df['tags'].apply(stem)

/tmp/ipykernel_6951/3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [164]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features=5000,stop_words='english')

In [165]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [166]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [167]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [168]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [169]:
new_df.sample(5)

,id,title,tags
3275,8293,Howards End,merchant ivory’ adapt of em forster’ classic 1...
3257,11596,New Nightmare,freddy' back … and he' badder than ever! nancy...
2620,57943,The Love Letter,20th centuri comput game design scott exchang ...
3801,54580,Mondays in the Sun,after the closur of their shipyard in northern...
411,22538,Scott Pilgrim vs. the World,scott pilgrim is a film adapt of the critic ac...


In [171]:
new_df.shape

(4806, 3)

In [174]:
from sklearn.metrics.pairwise import cosine_similarity

In [176]:
similarity = cosine_similarity(vectors)

In [179]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [185]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [188]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [189]:
import pickle

In [194]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [193]:
similarity.shape

(4806, 4806)